In [1]:
#The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

from numpy import rad2deg,deg2rad,pi,sqrt,add,array,average
from healpy import ang2vec, newvisufunc
import burstutils as bf
from random import gauss
import statistics as s
import matplotlib.pyplot as plt

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = deg2rad(self.tiltB)
            self.tiltC = self.tiltA = deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0 , 0]
        self.bg = background


        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*pi/2 ]
    @property
    def normA(self):
        return  ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, samples,test=True,talk=False):   #is this how I inherit? 

    #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        

        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        
        if test:
            sample = 1
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 0
            topA = 1000
            ntheta = 10   #over sky chi points
            nphi = 37
            nA = 100

        else:
            sample = len(GRB.sourceangs) 
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 400
            topA = 1000
            ntheta = 31   #over sky chi points
            nphi = 120
            nA = 12
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            if talk:
                print("Testing " + str(rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                xA = bf.look_up_A(self.normA,sourcexyz)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
                dtheoryA=GRB.Ao*bf.response(sepA,xA)  #still need to define strength, brb and gonna do that 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg
                unccountsA = sqrt(countsA)
                detactualA = gauss(countsA,unccountsA)  #there is a lot of noise present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                xB = bf.look_up_B(self.normB,sourcexyz)

                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
                dtheoryB=GRB.Ao*bf.response(sepB,xB)  
                    #still need to define strength, brb and gonna do that 

                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg 
                unccountsB = sqrt(countsB)
                detactualB = gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB
                


                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
                xC =  bf.look_up_C(self.normC,sourcexyz)
                dtheoryC=GRB.Ao*bf.response(sepC,xC)  #still need to define strength, brb and gonna do that 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = sqrt(countsC)
                detactualC = gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                

                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
                xD = bf.look_up_D(self.normD,sourcexyz)
                dtheoryD=GRB.Ao*bf.response(sepD,xD)  #still need to define strength, brb and gonna do that 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = sqrt(countsD)
                detactualD = gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                

                
                
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,A=True)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,B=True)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,C=True)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,D=True)
                
                chisquared = add(add(chiA,chiB),add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = ang2vec(deg2rad(thetaloc),deg2rad(philoc))
                locoffset = rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                
                locunc.append(locoffset)
                loop +=1
            if talk:
                print("Avg loc offset = " + str(average(locunc)) + " deg.")

            self.localizationerrors.append(s.mean(locunc))
        return self.localizationerrors


    def plotSkymap(self,skyvals):
        im = array(skyvals)
        newvisufunc.mollview(im,min=0, max=15,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True)
        plt.title('All Sky Localization Uncertainty for BurstCube set as ' + str(rad2deg(self.tilt)) + ' deg')  #should add something about design too! 

        plt.show()

In [2]:
import GRBgenerator
sky = GRBgenerator.Sky(NSIDE=4,strength=500)

cube1 = FastCube(dettilt=45,background=1000)

In [3]:
cube1.response2GRB(GRB=sky,samples=10,test=False,talk=True)

Testing [ 11.71585239  45.        ]
Avg loc offset = 8.00434542212 deg.
Testing [  11.71585239  135.        ]
Avg loc offset = 6.73642257911 deg.
Testing [  11.71585239  225.        ]
Avg loc offset = 6.21284880415 deg.
Testing [  11.71585239  315.        ]
Avg loc offset = 9.93090434583 deg.
Testing [ 23.55646431  22.5       ]
Avg loc offset = 25.4836586973 deg.
Testing [ 23.55646431  67.5       ]
Avg loc offset = 9.37994383052 deg.
Testing [  23.55646431  112.5       ]
Avg loc offset = 7.75631383442 deg.
Testing [  23.55646431  157.5       ]
Avg loc offset = 6.78068890289 deg.
Testing [  23.55646431  202.5       ]
Avg loc offset = 5.25937199664 deg.
Testing [  23.55646431  247.5       ]
Avg loc offset = 6.64499080967 deg.
Testing [  23.55646431  292.5       ]
Avg loc offset = 6.41200419429 deg.
Testing [  23.55646431  337.5       ]
Avg loc offset = 21.82551339 deg.
Testing [ 35.6590877  15.       ]
Avg loc offset = 35.7263537112 deg.
Testing [ 35.6590877  45.       ]
Avg loc offset =

[8.0043454221192594,
 6.7364225791124568,
 6.2128488041505738,
 9.9309043458292834,
 25.483658697284309,
 9.3799438305244802,
 7.7563138344189566,
 6.7806889028913027,
 5.2593719966449672,
 6.6449908096716568,
 6.4120041942856689,
 21.825513389979751,
 35.726353711151027,
 29.266763314575101,
 17.527513491538706,
 5.208988014663583,
 6.0347554875272618,
 6.007087727076839,
 5.2583617519829273,
 5.6137223551322082,
 4.6866500748431656,
 18.532104251223529,
 27.846708627746708,
 34.580246635679003,
 47.82732464617564,
 40.464828381636913,
 31.02049148069738,
 54.336131036794079,
 47.186713697371246,
 4.49892434817931,
 3.5084188871684301,
 47.108583586511891,
 49.69191897628022,
 4.6240353791834492,
 5.2277691458945261,
 47.489621775706375,
 52.347357176267664,
 33.241892879738927,
 41.074149845640889,
 47.591992360824683,
 62.791527729865372,
 55.964316323473135,
 69.300767506737699,
 67.520881545190832,
 64.213104419076359,
 55.096642169653869,
 67.445081818555494,
 57.158189443676051,

In [ ]:
%debug

What still needs to be done:

Quad solvers all need to be refactored
There's a unique response function designed for array inputs, it needs to use the lookup table inside the chi squared solver where chisource xyz is the input in it and it'll figure out what response to use for each spot.

In [ ]:
import burstutils as b

In [ ]:
b.look_up_A([[0,0,1],[0,0,1]],[[0,0,1],[0,0,1]])

In [ ]:
eh = [[0,0,1],[0,0,1],[0,0,1],[0,0,1]]
hehe = [0.1,2,3]

In [ ]:
np.shape(hehe)[0]

In [ ]:
import numpy as np

In [ ]:
len(eh)

In [ ]:
(type(np.array([])))

In [ ]:
type(type(eh))

In [2]:
import numpy as np
import matplotlib.pyplot as plt